In [1]:
import os
import re
import pandas as pd
from pathlib import Path
from datetime import datetime

# Combine the data and remove sub-headers

In [ ]:
# === Merge ALL TrackMate spot tables into one clean dataset ===================
# - Recursively reads ../data/raw/**/*.csv|xlsx|xls
# - Skips TrackMate's 2 extra header rows (friendly names + units)
# - Cleans column names (UPPER_SNAKE)
# - Removes any repeated in-body sub-headers (e.g., "(pixels)", "(counts)")
# - Appends filename metadata
# - Converts numeric columns where possible
# - Saves to ../data/processed/combined_raw_data.csv (safe overwrite)
# ============================================================================
BASE = Path("..")  # notebooks/ as CWD
RAW_DIR = BASE / "data" / "raw"
PROC_DIR = BASE / "data" / "processed"
PROC_DIR.mkdir(parents=True, exist_ok=True)

OUT_CSV = PROC_DIR / "combined_raw_data.csv"

def parse_meta_from_name(p: Path) -> dict:
    stem = p.stem
    parts = stem.split("_")
    meta = {
        "experiment_id": stem,
        "batch":  parts[0] if len(parts) > 0 else None,
        "sample": parts[1] if len(parts) > 1 else None,
        "channel": parts[2] if len(parts) > 2 else None,
        "time_start": None,
        "time_end": None,
        "source_file": p.name,
    }
    for t in parts:
        m = re.fullmatch(r"(\d+)-(\d+)", t)
        if m:
            meta["time_start"], meta["time_end"] = int(m.group(1)), int(m.group(2))
            break
    return meta

def clean_columns(cols) -> list:
    return (
        pd.Index(cols).astype(str)
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[()]", "", regex=True)
        .str.upper()
        .tolist()
    )

def read_trackmate_table(p: Path) -> pd.DataFrame:
    # Keep row-1 header; skip the 2 extra rows (friendly names + units)
    if p.suffix.lower() == ".csv":
        df = pd.read_csv(p, skiprows=[1, 2])
    else:
        df = pd.read_excel(p, header=0, skiprows=[1, 2])
    df.columns = clean_columns(df.columns)

    # Identify key numeric columns commonly present in TrackMate exports
    key_numeric_candidates = [
        "QUALITY", "FRAME", "POSITION_X", "POSITION_Y", "POSITION_Z",
        "RADIUS", "MEAN_INTENSITY", "MEDIAN_INTENSITY", "MIN_INTENSITY",
        "MAX_INTENSITY", "TOTAL_INTENSITY", "STD_INTENSITY", "SNR_CH1"
    ]
    keys = [c for c in key_numeric_candidates if c in df.columns]

    # Coerce keys to numeric (sub-header rows will turn into NaN)
    for c in keys:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Drop repeated in-body sub-headers:
    #  - any row where *all* key numeric columns are NaN
    #  - or where a key column literally matches strings like "quality", "(pixels)", etc.
    if keys:
        mask_all_nan = df[keys].isna().all(axis=1)
    else:
        mask_all_nan = pd.Series(False, index=df.index)

    suspicious_tokens = {"quality", "pixels", "frames", "counts"}
    mask_token = pd.Series(False, index=df.index)
    for c in df.columns:
        if df[c].dtype == "object":
            mask_token |= df[c].str.lower().isin(suspicious_tokens).fillna(False)

    df = df[~(mask_all_nan | mask_token)].reset_index(drop=True)

    return df

def safe_write_csv(df: pd.DataFrame, dst: Path) -> Path:
    try:
        if dst.exists():
            os.remove(dst)  # avoid Windows lock issues
        df.to_csv(dst, index=False)
        return dst
    except PermissionError:
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        alt = dst.with_name(dst.stem + f"_{ts}" + dst.suffix)
        df.to_csv(alt, index=False)
        print(f"[WARN] PermissionError writing {dst}. Saved as: {alt}")
        return alt

# Collect files
paths = []
for ext in ("*.csv", "*.CSV", "*.xlsx", "*.xls"):
    paths += list(RAW_DIR.rglob(ext))
paths = sorted(set(paths))
if not paths:
    raise FileNotFoundError(f"No files found under {RAW_DIR}")

# Read + attach metadata
frames, failed = [], []
for p in paths:
    try:
        df = read_trackmate_table(p)
        meta = parse_meta_from_name(p)
        for k, v in meta.items():
            df[k.upper()] = v  # store meta in UPPERCASE
        frames.append(df)
    except Exception as e:
        failed.append((str(p), str(e)))

combined = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()





C:\Users\Yuhan_Li\AppData\Local\Temp\ipykernel_54204\1120478200.py:123: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  combined[c] = pd.to_numeric(combined[c], errors="ignore")


[WARN] PermissionError writing ..\data\processed\combined_raw_data.csv. Saved as: ..\data\processed\combined_raw_data_20250911_130311.csv
Files read: 347 | Failed: 0
Combined shape: (62733, 54)
Saved CSV to: ..\data\processed\combined_raw_data_20250911_130311.csv


In [51]:
# Display the first few rows to understand the structure and content.
print("\n--- First 5 Rows of the Dataset ---")
combined.head()


--- First 5 Rows of the Dataset ---


,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,...,MEAN_INTENSITY_CH3,MEDIAN_INTENSITY_CH3,MIN_INTENSITY_CH3,MAX_INTENSITY_CH3,TOTAL_INTENSITY_CH3,STD_INTENSITY_CH3,CONTRAST_CH2,SNR_CH2,CONTRAST_CH3,SNR_CH3
0,ID2945,2945.0,0.0,209.0,145.948467,95.621082,0.0,3.0,3.0,8.110457,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID2946,2946.0,0.0,220.0,148.047821,94.433312,0.0,7.0,7.0,8.321153,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID2947,2947.0,0.0,195.0,145.259303,91.481847,0.0,11.0,11.0,7.834106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID2948,2948.0,0.0,234.0,145.152217,91.656074,0.0,21.0,21.0,8.581834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ID2951,2951.0,0.0,219.0,143.997185,89.188898,0.0,12.0,12.0,8.302220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# Display basic descriptive statistics for numerical columns.
print("\n--- Descriptive Statistics for Numerical Columns ---")
combined.describe()


--- Descriptive Statistics for Numerical Columns ---


,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,VISIBILITY,...,MEAN_INTENSITY_CH3,MEDIAN_INTENSITY_CH3,MIN_INTENSITY_CH3,MAX_INTENSITY_CH3,TOTAL_INTENSITY_CH3,STD_INTENSITY_CH3,CONTRAST_CH2,SNR_CH2,CONTRAST_CH3,SNR_CH3
count,62733.000000,62733.000000,62733.000000,62733.000000,62733.000000,62733.0,62733.000000,62733.000000,62733.000000,62733.0,...,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,44793.245453,15.117227,177.698532,99.390484,100.097783,0.0,11.101860,11.101860,5.216079,1.0,...,402.777655,387.875000,96.000000,855.312500,75793.062500,161.005977,-0.248298,-1.378942,0.013047,-0.005739
std,26670.920014,15.171854,71.971663,59.471399,62.214191,0.0,7.894153,7.894153,1.087667,0.0,...,80.357446,86.911353,48.751068,106.773729,30055.699316,31.165775,0.120335,1.456640,0.169850,1.246837
min,2932.000000,0.000000,1.000000,0.994375,1.604550,0.0,0.000000,0.000000,0.280506,1.0,...,170.186047,154.000000,48.000000,536.000000,14636.000000,80.246754,-0.577982,-6.515443,-0.506486,-4.353062
25%,14569.000000,4.000000,128.000000,52.132938,54.212153,0.0,5.000000,5.000000,4.456830,1.0,...,375.752828,397.250000,52.750000,813.250000,56332.250000,146.172886,-0.291944,-1.341738,-0.016618,-0.070357
50%,46856.000000,10.000000,167.000000,86.252984,85.515696,0.0,10.000000,10.000000,5.073308,1.0,...,435.333063,419.000000,86.500000,880.000000,81868.000000,159.519929,-0.228579,-1.120674,0.062652,0.295681
75%,67938.000000,21.000000,214.000000,136.713200,130.784498,0.0,16.000000,16.000000,5.842186,1.0,...,452.081915,445.250000,122.250000,905.000000,95040.000000,182.593692,-0.178541,-0.705161,0.112496,0.647224
max,88245.000000,97.000000,770.000000,361.456676,387.718350,0.0,37.000000,37.000000,8.907281,1.0,...,477.508380,451.000000,185.000000,1000.000000,131193.000000,218.677388,-0.093763,-0.352622,0.203212,0.969804


# Missing value

In [ ]:
# Check for missing values in each column
print("\n--- Missing Values Before Cleaning ---")
print(combined.isnull().sum())



--- Missing Values Before Cleaning ---
LABEL                       0
ID                          0
TRACK_ID                    0
QUALITY                     0
POSITION_X                  0
POSITION_Y                  0
POSITION_Z                  0
POSITION_T                  0
FRAME                       0
RADIUS                      0
VISIBILITY                  0
MANUAL_SPOT_COLOR       62733
MEAN_INTENSITY_CH1          0
MEDIAN_INTENSITY_CH1        0
MIN_INTENSITY_CH1           0
MAX_INTENSITY_CH1           0
TOTAL_INTENSITY_CH1         0
STD_INTENSITY_CH1           2
CONTRAST_CH1                0
SNR_CH1                     2
ELLIPSE_X0                  0
ELLIPSE_Y0                  0
ELLIPSE_MAJOR               0
ELLIPSE_MINOR               0
ELLIPSE_THETA               0
ELLIPSE_ASPECTRATIO         2
AREA                        0
PERIMETER                   0
CIRCULARITY                 0
SOLIDITY                    0
SHAPE_INDEX                 0
EXPERIMENT_ID               0


In [62]:
import re
import pandas as pd
from pathlib import Path

ROOT = Path("..")
INPUT  = ROOT / "data/processed/combined_cleaned.csv"
OUTPUT = ROOT / "data/processed/combined_cleaned.csv"   # 覆盖原文件（如需保留旧版改成别的名字）

df = pd.read_csv(INPUT)
print("Original shape:", df.shape)


na_rate = df.isna().mean()
to_drop_by_na = na_rate[na_rate > 0.99].index.tolist()


ch23_cols = [c for c in df.columns if re.search(r'(?:^|_)CH[23](?:_|$)', c)]

manual_drop = [c for c in ["MANUAL_SPOT_COLOR"] if c in df.columns]

to_drop = sorted(set(to_drop_by_na + ch23_cols + manual_drop))
print(f"Drop {len(to_drop)} columns:", to_drop)

df = df.drop(columns=to_drop, errors="ignore")

# ===== B) 删除极少量 NA 行（STD_INTENSITY_CH1 或 SNR_CH1 为空）=====
tiny_na_cols = [c for c in ["STD_INTENSITY_CH1", "SNR_CH1"] if c in df.columns]
if tiny_na_cols:
    na_rows_mask = df[tiny_na_cols].isna().any(axis=1)
    print(f"Rows to drop due to NA in {tiny_na_cols}: {int(na_rows_mask.sum())}")
    df = df.loc[~na_rows_mask].reset_index(drop=True)

print("New shape:", df.shape)

# ===== C) 快速复查剩余缺失率（Top 20）=====
missing_pct = ((df.isna().sum()/len(df))*100).round(2).sort_values(ascending=False).head(20)
print("\nTop-20 missing % after cleaning:\n", missing_pct)

# ===== D) 保存 =====
OUTPUT.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUTPUT, index=False)
print("Saved ->", OUTPUT.resolve())


C:\Users\Yuhan_Li\AppData\Local\Temp\ipykernel_2192\383640072.py:9: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT)


Original shape: (62731, 53)
Drop 16 columns: ['CONTRAST_CH2', 'CONTRAST_CH3', 'MAX_INTENSITY_CH2', 'MAX_INTENSITY_CH3', 'MEAN_INTENSITY_CH2', 'MEAN_INTENSITY_CH3', 'MEDIAN_INTENSITY_CH2', 'MEDIAN_INTENSITY_CH3', 'MIN_INTENSITY_CH2', 'MIN_INTENSITY_CH3', 'SNR_CH2', 'SNR_CH3', 'STD_INTENSITY_CH2', 'STD_INTENSITY_CH3', 'TOTAL_INTENSITY_CH2', 'TOTAL_INTENSITY_CH3']
Rows to drop due to NA in ['STD_INTENSITY_CH1', 'SNR_CH1']: 0
New shape: (62731, 37)

Top-20 missing % after cleaning:
 CHANNEL                3.44
TIME_END               3.23
TIME_START             3.23
LABEL                  0.00
PERIMETER              0.00
ELLIPSE_MAJOR          0.00
ELLIPSE_MINOR          0.00
ELLIPSE_THETA          0.00
ELLIPSE_ASPECTRATIO    0.00
AREA                   0.00
SOLIDITY               0.00
CIRCULARITY            0.00
ELLIPSE_X0             0.00
SHAPE_INDEX            0.00
EXPERIMENT_ID          0.00
BATCH                  0.00
SAMPLE                 0.00
ELLIPSE_Y0             0.00
SNR_CH1     

PermissionError: [Errno 13] Permission denied: '..\\data\\processed\\combined_cleaned.csv'

In [61]:
# Double check the missing value
((df.isna().sum() / len(df) * 100).round(2)
 .sort_values(ascending=False).head(20))


CHANNEL                3.44
TIME_END               3.23
TIME_START             3.23
LABEL                  0.00
PERIMETER              0.00
ELLIPSE_MAJOR          0.00
ELLIPSE_MINOR          0.00
ELLIPSE_THETA          0.00
ELLIPSE_ASPECTRATIO    0.00
AREA                   0.00
SOLIDITY               0.00
CIRCULARITY            0.00
ELLIPSE_X0             0.00
SHAPE_INDEX            0.00
EXPERIMENT_ID          0.00
BATCH                  0.00
SAMPLE                 0.00
ELLIPSE_Y0             0.00
SNR_CH1                0.00
ID                     0.00
dtype: float64

In [58]:
print(df.isnull().sum())

LABEL                       0
ID                          0
TRACK_ID                    0
QUALITY                     0
POSITION_X                  0
POSITION_Y                  0
POSITION_T                  0
FRAME                       0
RADIUS                      0
VISIBILITY                  0
MEAN_INTENSITY_CH1          0
MEDIAN_INTENSITY_CH1        0
MIN_INTENSITY_CH1           0
MAX_INTENSITY_CH1           0
TOTAL_INTENSITY_CH1         0
STD_INTENSITY_CH1           0
CONTRAST_CH1                0
SNR_CH1                     0
ELLIPSE_X0                  0
ELLIPSE_Y0                  0
ELLIPSE_MAJOR               0
ELLIPSE_MINOR               0
ELLIPSE_THETA               0
ELLIPSE_ASPECTRATIO         0
AREA                        0
PERIMETER                   0
CIRCULARITY                 0
SOLIDITY                    0
SHAPE_INDEX                 0
EXPERIMENT_ID               0
BATCH                       0
SAMPLE                      0
CHANNEL                     0
TIME_START

In [57]:
df = df.drop(columns=["POSITION_Z"], errors="ignore")
df.head()

,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_T,FRAME,RADIUS,VISIBILITY,...,MEAN_INTENSITY_CH3,MEDIAN_INTENSITY_CH3,MIN_INTENSITY_CH3,MAX_INTENSITY_CH3,TOTAL_INTENSITY_CH3,STD_INTENSITY_CH3,CONTRAST_CH2,SNR_CH2,CONTRAST_CH3,SNR_CH3
0,ID2945,2945.0,0.0,209.0,145.948467,95.621082,3.0,3.0,8.110457,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID2946,2946.0,0.0,220.0,148.047821,94.433312,7.0,7.0,8.321153,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID2947,2947.0,0.0,195.0,145.259303,91.481847,11.0,11.0,7.834106,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID2948,2948.0,0.0,234.0,145.152217,91.656074,21.0,21.0,8.581834,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ID2951,2951.0,0.0,219.0,143.997185,89.188898,12.0,12.0,8.302220,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
